<a href="https://colab.research.google.com/github/telpirion/FantasyMaps/blob/main/notebooks/final/1_firestore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Storing training data in Firestore

{TODO: Update the links below.}

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/notebook_template.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/notebook_template.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

## Overview

This notebook demonstrates how to collect data from social media (e.g. Reddit), preprocess that data, and then store that data in a Firestore collection. In this scenario, you collect fictional, hand-drawn maps that are used for virtual role-playing games. (Later, you convert these maps into training data for a Vertex AI notebook.)


### Objective

In this tutorial, you learn how to create a dataset in Cloud Storage and Firestore from a third-party API.

This tutorial uses the following Google Cloud resources:

+ Cloud Storage bucket
+ Firestore collection

The steps performed include:

1. Collecting images from Reddit
1. Storing the images in Cloud Storage
1. Inferring training data from the images
1. Storing that training data in a Firestore collection

### Dataset

In this tutorial, you collect data from subreddit posts on Reddit. The individual posts are processed and their metadata stored in Firestore. Any image data the posts contain are extracted and stored in a Storage bucket.

### Costs

This tutorial uses billable components of Google Cloud:

* Firestore
* Cloud Storage

Learn about [Firestore pricing](https://cloud.google.com/firestore/pricing),
and [Cloud Storage pricing](https://cloud.google.com/storage/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [18]:
%%writefile requirements.txt
google-cloud-firestore
google-cloud-secret-manager
google-cloud-storage
praw
pandas
numpy
spacy
pillow

Writing requirements.txt


In [19]:
! pip install --upgrade -qr requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.2/326.2 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.6/184.6 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 104.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is incompatible.


We will also use a simple natural language parsing library to analyze posts. For this use case, we'll use the open source library [spaCy](https://spacy.io). spaCy requires that a language model be downloaded before it can be used.

In [20]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


We have prepared a library of functions to help you process images.

In [21]:
! pip uninstall -y fantasy_maps_lib
! pip install "git+https://github.com/telpirion/fantasy-maps-lib.git#egg=fantasy_maps_lib"

  Cloning https://github.com/telpirion/fantasy-maps-lib.git to /tmp/pip-install-1207w0to/fantasy-maps-lib_992463cea1e94686922e9fa17da42a92
  Running command git clone --filter=blob:none --quiet https://github.com/telpirion/fantasy-maps-lib.git /tmp/pip-install-1207w0to/fantasy-maps-lib_992463cea1e94686922e9fa17da42a92
  Resolved https://github.com/telpirion/fantasy-maps-lib.git to commit 71c9abc6dec9f44a001ef9c89fac7f9336358275
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 45.5 MB/s eta 0:00:00
  Created wheel for fantasy_maps_lib: filename=fantasy_maps_lib-0.0.0-py3-none-any.whl size=17158 sha256=35b686da5d7c14e15d13d49efc260682c715da1f8032edd22b51d55005c98d99
  Stored in directory: /tmp/pip-ephem-wheel-cache-5pf8ynaf/wheels/95/85/a3/5ea513fa633d19614cc7ba22d8012af

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Firestore API](https://console.cloud.google.com/flows/enableapi?apiid=firestore.googleapis.com).

4. [Enable the Secret Manager API](https://console.cloud.google.com/flows/enableapi?apiid=secretmanager.googleapis.com).

5. [Enable the Cloud Storage API](https://console.cloud.google.com/flows/enableapi?apiid=storage.googleapis.com).

6. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
# If using Colab secrets, import them here
from google.colab import userdata
PROJECT_ID = userdata.get('PROJECT_ID')

! gcloud config set project {PROJECT_ID}

In [22]:
# If running in Vertex Workbench or Colab Enterprise, set your project here
PROJECT_ID = !gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
print(PROJECT_ID)

fantasymaps-334622


In [ ]:
# If running in some other Jupyter notebook installation, provide your
# Google Cloud project name
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [23]:
REGION = "us-west1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [ ]:
from google.colab import auth
auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.


In [24]:
BUCKET_URI = f"gs://fantasy-maps-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### Import libraries

In [25]:
from google.cloud import firestore
from google.cloud import secretmanager
from google.cloud import storage

from PIL import Image

import hashlib
import json
import math
import numpy as np
import os
import pandas as pd
import pprint
import praw
import re
import requests
import shutil
import spacy

In [26]:
from fantasy_maps.reddit import posts
from fantasy_maps.image import extract, shards
from fantasy_maps.image.image_metadata import ImageMetadata

### Get a Reddit API key

You need a [Reddit API key](https://www.reddit.com/wiki/api/) to access Reddit programmatically. Copy your API key into the labeled fields of the dictionary in the following cell.

**Note**: Once you have an API key, you must store it somewhere safe. It is recommended to store your API key as a JSON-formatted string in Cloud Secret Manager.

In [27]:
SECRET_RESOURCE_NAME = f"projects/{PROJECT_ID}/secrets/reddit-api-key/versions/1" # @param {type:"string"}

In [28]:
secret_client = secretmanager.SecretManagerServiceClient()
secret = secret_client.access_secret_version(name=SECRET_RESOURCE_NAME)
reddit_key_json = json.loads(secret.payload.data)
print(reddit_key_json)

{'secret': '_XDRI2jgcVAJ6xKIWmA46yz8CZw', 'client_id': 'Z0g7xbmKNB9Mew', 'user_agent': 'script:ScrapeForNLP:v1.0 (by u/Telpirion-78)', 'user_name': 'Telpirion-78'}


## Query data (posts) on Reddit

Now that we have our API key ready for use, we can query Reddit for our data! In the next cell, we will read the top 100 "hot" posts from a subreddit.

For our use-case, we want to check the posts to see whether: 1) they have an image associated with them; and 2) the title gives us some clues as to the contents (e.g. columns and rows) contained in the image.

In [29]:
columns = ['Title', 'Post', 'ID', 'URL']

subreddit_name = "battlemaps"
reddit_posts = posts.get_reddit_posts(reddit_credentials=reddit_key_json,
                                subreddit_name=subreddit_name, limit=100)
reddit_dicts = posts.convert_posts_to_dicts(reddit_posts, columns)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Now that we have the top 100 "hot" posts from the subreddit, we're going to filter for only the posts that we want. Again, our criteria are: 1) must have an image; 2) the title must have the grid dimensions of the image.

We'll use Pandas to visualize the resulting data.

In [30]:
reddit_images = [
  ImageMetadata(
      url=d['URL'],
      title=d['Title'],
      rid=d['ID']) for d in reddit_dicts if (
    'jpeg' in d['URL'] and
    re.search(r'\d+x\d+', d['Title'])
  )
]
print(len(reddit_images))

41


## Process the images and their metadata

In this next step, we will process all of the Reddit posts with images posts:

1. Download the image itself
2. Parse each image's metadata
3. (If needed) Split the image into smaller images
4. Store the metadata in Firestore
5. Save the images on Cloud Storage

### Download the image

We're going to download the image locally. We'll need a meaningful filename to save the image. We also need a directory system to save the image to.

We also want to avoid downloading the same image more than once. We'll need to compare the images programmatically to verify that each image is unique.

The easiest way to do this will be to reduce each image to a unique hash value and then ensure that we never have two copies of the same hash value. For the sake of simplicity, we'll use these hash values as the unique ID for each image.

In [ ]:
# If using Vertex Workbench or a local Jupyter notebook, run this cell.
root_dir = os.getcwd()

In [ ]:
# If using Colab (not Colab Enterprise), mount Google Drive
root_dir = '/content/drive'
from google.colab import drive
drive.mount(root_dir)

In [14]:
# If using Colab Enterprise, use gcsfuse
root_dir = '/content/drive'

!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update && apt -qq install gcsfuse
!gcsfuse --implicit-dirs $BUCKET_URI $root_dir

deb https://packages.cloud.google.com/apt gcsfuse-jammy main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: apt-key is deprecated. Manage keyring files in trusted.gpg.d instead (see apt-key(8)).
100  1021  100  1021    0     0  12527      0 --:--:-- --:--:-- --:--:-- 12604
OK
75 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://packages.cloud.google.com/apt/dists/gcsfuse-jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 n

In [34]:
BUCKET_NAME = BUCKET_URI.replace('gs://', '')
root_dir = '/gcs/content/'

if not os.path.exists(root_dir):
  os.makedirs(root_dir, exist_ok=True)

!gcsfuse --implicit-dirs {BUCKET_NAME} {root_dir}

{"timestamp":{"seconds":1726690568,"nanos":144119077},"severity":"INFO","message":"Start gcsfuse/2.4.1 (Go version go1.23.0) for app \"\" using mount point: /gcs/content/\n"}
{"timestamp":{"seconds":1726690568,"nanos":144163329},"severity":"INFO","message":"GCSFuse config","config":{"debug":{},"file-cache":{"download-chunk-size-mb":50,"max-parallel-downloads":16,"max-size-mb":-1,"parallel-downloads-per-file":16,"write-buffer-size":4194304},"file-system":{"dir-mode":"755","file-mode":"644","gid":-1,"ignore-interrupts":true,"uid":-1},"gcs-auth":{"reuse-token-from-url":true},"gcs-connection":{"client-protocol":"http1","grpc-conn-pool-size":1,"limit-bytes-per-sec":-1,"limit-ops-per-sec":-1,"max-idle-conns-per-host":100,"sequential-read-size-mb":200},"gcs-retries":{"max-retry-sleep":30000000000,"multiplier":2},"implicit-dirs":true,"list":{},"logging":{"log-rotate":{"backup-file-count":10,"compress":true,"max-file-size-mb":512},"severity":"INFO"},"metadata-cache":{"deprecated-stat-cache-capa

In [35]:
local_reddit_data_dir = f'{root_dir}reddit_maps_data'

if os.path.exists(local_reddit_data_dir):
  shutil.rmtree(local_reddit_data_dir)

os.mkdir(local_reddit_data_dir)

hashes = []

for post in reddit_images:
    url = post.url
    filename = posts.make_nice_filename(post.title)

    if filename == '':
        continue

    path = f'{local_reddit_data_dir}/{filename}'
    uid = extract.download_image_local(url=url, path=path)
    if uid in hashes:
      continue

    hashes.append(uid)

    post.path = path
    post.uid = uid

In [36]:
pprint.pprint(reddit_images[0].to_dict())

{'bboxes': [],
 'cell_height': 0,
 'cell_offset_x': 0,
 'cell_offset_y': 0,
 'cell_width': 0,
 'columns': 1,
 'height': 0,
 'path': '/gcs/content/reddit_maps_data/plane_players_portal_brand_new_czepeku.31x60.jpg',
 'rid': '1fjqahc',
 'rows': 1,
 'title': 'Which plane will your players visit through the portal? - Brand New '
          'Czepeku Scene/Battlemap Combo! [31x60]',
 'uid': 'c60cc1bafc2e1e94e34dbdd05f8c67203cefd011',
 'url': 'https://i.redd.it/885viw3mxjpd1.jpeg',
 'width': 0}


### Parse the image for metadata

All we know from the Reddit API is that these posts have JPGs associated with them and that they contain a substring in the format "NNxNN." However, we need more than just images and rough columns and rows for training a Vertex AI AutoML image object detection model. We would even need more data just to use these images in a VTT app.

To get valid training data and VTT data, we need to make some inferences about the images based upon the data that we have (or can get). The data we have are the number of columns and rows stated in the post's title (granted, these are not always accurate). The data we can get is the image's width and height. From these four data points, and assuming that they are accurate and that all cells in the map are uniform, we can infer the width and height of cells in the map.

Using the cell width and height, we can compute the rest of the data required for both an image object detection model and the data needed for a VTT app.

#### Virtual tabletop (VTT) data

The easiest for us to compute is the VTT data. This data provides us with the `cellWidth` and `cellHeight` data that will allow us to complete the ML training data.

The JSON structure of VTT data is:

```json
{
    "imageHeight": ##,
    "imageWidth": ##,
    "cellHeight": ##,
    "cellWidth": ##,
    "cellOffsetY": ##,
    "cellOffsetX": ##
}

```


#### Image object detection training data

AutoML image object detection on Vertex AI requires a JSONL file with information for the training data. Each line in the JSONL file needs to contain: the Cloud Storage URI of the image; and the bounding boxes of the objects (cells) that we want to train the model to identify on the image.

**Tip**: The format we use for training data is called the [Common Objects in Context (COCO)](https://cocodataset.org/#format-data), with some notable exceptions. The canonical example expresses the maximum values as `width` and `height`, where as we will annotate our data as `x` and `y` values expressed as a percentage of the total height and widthy of the image.

The structure of the JSON data in the JSONL file is:

```json
{
    "imageGcsUri": "URI",
    "boundingBoxAnnotations": {
        "displayName": "LABEL_NAME",
        "xMin": ##,
        "xMax": ##,
        "yMin": ##,
        "yMax": ##,
    }
}
```

For each bounding box, we need to provide a percentage value that expresses the vertices of the bounding box as a set of x and y pairs. Also, each bounding box needs to be given a label for that bounding box; all of our bounding boxes are "cells" so each one gets the label `cell`.

**Note**: A training image in Vertex AI can only have at most 500 bounding boxes; many fantasy maps have many more than 500 cells. So that we can use the most of the training data, we will split too large images until smaller images, or "shards", and use the shards  for training. We'll create the shards a bit later.

You can read more about how to format your training manifest JSONL file in [the documentation](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image#object-detection).

In [49]:
import sys
test_path = '/gcs/content/reddit_maps_data/plane_players_portal_brand_new_czepeku.31x60.jpg'
with open(test_path, 'rb') as f:
    img_bytes = f.read()
    img_size = sys.getsizeof(img_bytes)
    print(img_size)
    img = Image.frombytes(mode='YCbCr', size=img_size, data=img_bytes)
    img.show()

#img = Image.open(test_path)
#img.show()

1547651


ValueError: Size must be a tuple

In [38]:
for post in reddit_images:
    if not os.path.exists(post.path):
      continue

    local_path = post.path

    # Get width & height for original image
    w, h = extract.get_image_width_and_height(local_path)

    post.width = w
    post.height = h

    # Get columns & rows for original image, based upon the name.
    paths = local_path.split(".")
    dims = paths[-2]
    cols, rows = dims.split("x")

    cols = int(cols)
    rows = int(rows)

    post.columns = cols
    post.rows = rows

    # If image doesn't need to be sharded, simply compute and continue
    if (cols * rows) <= 500:
        bboxes = extract.compute_bboxes(img_metadata=post)
        post.bboxes = bboxes
        continue

UnidentifiedImageError: cannot identify image file '/gcs/content/reddit_maps_data/plane_players_portal_brand_new_czepeku.31x60.jpg'

In [ ]:
NUM_SHARDS=3
SHARD_COLS=15
SHARD_ROWS=15
    # Compute the number of shards
    img_metadata = ImageMetadata(path=local_path,
                                 width=w,
                                 height=h,
                                 columns=cols,
                                 rows=rows,
                                 cell_width=vtt["cellWidth"],
                                 cell_height=vtt["cellHeight"])
    shards = shards.compute_shard_coordinates(img_metadata=img_metadata,
                                              num_shards=NUM_SHARDS,
                                              shard_cols=SHARD_COLS,
                                              shard_rows=SHARD_ROWS)
    print(row)
    print(len(shards))
    for shard in shards:
        print(local_path)
        print(shard)
        shard_df = extract.create_shard(x_min=shard[0], y_min=shard[1], x_max=shard[2],
                                   y_max=shard[3], cols=shard[4], rows=shard[5],
                                   img_path=local_path, parent_id=row["UID"])

        if shard_df is None:
            continue

        s_vtt = vtt
        s_vtt["width"] = int(shard_df.iloc[0]["Width"])
        s_vtt["height"] = int(shard_df.iloc[0]["Height"])
        shard_df.at[0, "VTT"] = json.dumps(s_vtt)

        bboxes = extract.compute_bboxes(dataframe=shard_df,
                                cell_width=vtt["cellWidth"],
                                cell_height=vtt["cellHeight"])

        shard_df.at[0, "BBoxes"] = json.dumps({ "bboxes": bboxes })
        shards_df = pd.concat([shards_df, shard_df])

Finally, now that we have all of the VTT and image object detection metadata computed for the original images and/or shards, we can join the two dataframes together into one. As part of this process, we also want to reindex the resulting dataframe so that it uses the UIDs we calculated instead of the automatically generated indices.

In [ ]:
complete_df = pd.concat([jpg_df, shards_df])
complete_df.set_index("UID", inplace=True)
complete_df.fillna("", inplace=True)
complete_df.head(10)

## Store the images in Google Cloud Storage

Now that we've created the image shards, we can begin uploading the images to Google Cloud Storage. We'll need to have a Storage bucket already created for this next cell of code.

In [ ]:
for i, row in complete_df.iterrows():
    gcs_uri = store_image_gcs(project_id=PROJECT_ID, series=row,
                                bucket_name=BUCKET, prefix="FantasyMapsTest")
    complete_df.at[i, "GCS URI"] = gcs_uri

## Store the metadata in Firestore

Next we're going to store all of this metadata and URI in Firestore. The benefit of using Firestore is that the fields with JSON-formatted strings-`VTT` and `BBoxes` will automatically be translated into the correct document structure in Firestore after they've been upserted.

Very, very last step: iterate over all the training data and store the metadata in the Firestore collection.

In [ ]:
for uid, row in complete_df.iterrows():
    store_metadata_fs(project_id=PROJECT_ID, series=row,
                      collection_name=COLLECTION_NAME, uid=uid)


## Check the results of the metadata creation

Now that (hopefully) all of the image metadata has been added to the Firestore collection, we can review the data to ensure that it is correct.

To do this, we'll review the documents stored in the Firestore collection to verify that it has all the data we need--the VTT data, bounding boxes, and the GCS URI of the image.

In [ ]:
client = firestore.Client(project=PROJECT_ID)
collection = client.collection(COLLECTION_NAME)

docs = collection.where("BBoxes", "!=", "").select(field_paths=["gcsURI", "filename", "VTT", "Parent"]).stream()

With this Firestore query, we can verify the image metadata against the stored image in the Storage bucket. We'll first take the results of this query, compose it into a `pandas.DataFrame` object, and then print it out to the cell output. We can first take a look at the parent map (assuming that the map has been sharded) and then conclude whether the map and all its shards should be removed from the training set.

In [ ]:
docs_list = ((d.to_dict(), d.id) for d in docs)
docs_df = pd.DataFrame()
for i, d in enumerate(docs_list):
    d_dict = d[0]
    vtt = d_dict["VTT"]
    d_dict["VTT"] = json.dumps(vtt)
    d_dict["UID"] = d[1]
    docs_df = pd.concat([docs_df, pd.DataFrame(data=d_dict, index=[0])], ignore_index=True)

In [ ]:
docs_df.set_index("UID", inplace=True)
check_set_df = docs_df[["filename", "gcsURI", "Parent"]]
check_set_df.head(10)

Not everyone on Reddit follows the same conventions. Sometimes, there might be be a post where there are dimensions mentioned in the post (e.g. "50x40"), but the image doesn't actually have gridlines.

We shouldn't allow these images into the training and test dataset for our model. Unfortunately, we have to review the images that we've collected on GCS and then verify that they do (or don't!) have gridlines visually.


We'll start by printing out the entirety of our `DataFrame`.

In [ ]:
pd.set_option("display.max_rows", 1000)
check_set_df.sort_values(by="filename", ascending=True, inplace=True)
display(check_set_df)

This final step of data prepartion is to mark all of the unusable images in the Firestore collection. Luckily, we can use the Google Cloud Console to view the contents of our Storage bucket. We can even add new fields to the documents in our Firestore collection!

![Storage user interface in the Cloud Console](https://github.com/telpirion/FantasyMaps/blob/main/notebooks/final/resources/StorageUI.png?raw=1)
_Figure. The Google Cloud Storage user interface, showing images in a bucket._

![Firestore user interface in the Cloud Console](https://github.com/telpirion/FantasyMaps/blob/main/notebooks/final/resources/FirestoreUI.png?raw=1)
_Figure. The Cloud Firestore user interface, showing a "Usable" field being added to a document._

For this very last data preparation step, we will visually inspect all of the "parent" images in the Cloud Storage bucket. We will then create a list unusable images, where we store the image's UID. Finally, we will do a bulk update job to Firestore, setting a `Usable` field on the images to `False`.

In [ ]:
unusable_parent_images = [
    "d3ee0039aeff5c33de778c5adbdd000f21c0b4cd",
    "9a7e82433239b0087121f6fd31e133f5a94fa7dc",
    "fc70f330cd5a3aaa3be94e0d603dab2876f9fca1",
    "890c0d27318b0286aebf67c392fab286bdc4e7c5",
    "2f4f496466d6e9fb8ad9791ccb81f7c13fd407db",
    "4c0d3eb86ed599f496f7e30e18025021aebfe153",
    "4bf88b8acc669331a65465e8c4b37fd8b9495e4b",
    "b15fe8185c30b3e7800e42e280a3792998a0b55f",
    "5cd7bbd5882fe8b9a270be1aa911c0ba858e818d",
    "67925948371de53b58c09b32c97af60f72c58e0f",
    "a816b6eb31b71cad5b50531d6e18ef46cb451cbd",
    "f00be2ed7b19b8d0f44915a1886437193aa224aa",
    "890c0d27318b0286aebf67c392fab286bdc4e7c5",
    # Others ....
]

usable_parent_images = [
    "7a903be0bad0fc00bfabbefd682b6eef23263b67",
    "b1fe854b9c658cf9312319a447b594743513499b",
    "3a52781685d7aed530a685739b58341fafd2e721",
    "3d17d612a843af7a5ad1a2b2d5dcce29e67d367f",
    "aa33a7ed5c3c87147fd25dafe6c0a1d3eb29dbe5",
    "2ce9f80408137e36531581fb22ee3fe892f41f76",
    "e086c1cf420e27448bc1a45147b5c43df4b3d8d0",
    "5bc994d7d38c1fc4654c58666d674200f731986b",
    "820c3bbfe4d14694ecb729ce3f45b4bda031f61a",
    "2d323018c74db7e0432ff368283ea429f13bd36d",
    "343833ab1d8cd17dae6b702830864500d1e66e19",
    "c60b952d0c20a63dc263220ec5b49a54fd20d175",
    "8ac01d3a84bc548a8b243d08c6e031206f293908",
    "f00be2ed7b19b8d0f44915a1886437193aa224aa",
    "950390c88b7bd9d6f886e5f01bae9460c0aa407b",
    "3322c1b4795e4a9e4477feafd55685d647b4e29c",
    "34362be27ada680a58e42d26758bf08c01d3460a",
    "60d815ba2c1458c3a4039595a5ed723a7501a36e",
    "928b339ba222a3933fce4523f8033fa3eb7ed62f",
    "4931f0033f0ab217fd0fe2a2024d22a119782e2c",
    "83846604933daff160cefc28c2a828bd93a84e1a",
    "d2fe7281d8b1e043009033c957ca347847343e14",
    "c1552a0046ef4ece5d146544043edb2deb97f7a1",
    # And more ...
]

**Note**: If you're thinking that this manual process should be automatable--you're right! In another notebook, we will use a pre-trained version of our gridline-detecting model to accept or reject images.

In [ ]:
usable_set = set(usable_parent_images)
unusable_set = set(unusable_parent_images)

In [ ]:
firestore_client = firestore.Client(project=PROJECT_ID)
bulkwriter = firestore_client.bulk_writer()
collection = firestore_client.collection(COLLECTION_NAME)

In [ ]:
# Iterate over all of the metadata entries & images that we want to delete
unusable_shards = collection.where("Parent", "in", list(unusable_set)[:10]).stream()
for doc in unusable_shards:
    bulkwriter.delete(doc.reference)

bulkwriter.flush()

In [ ]:
# Iterate over all of the good entries
subset_start_index = 0
while subset_start_index < len(usable_set):
    subset = list(usable_set)[subset_start_index:subset_start_index + 10]
    usable_shards = collection.where("Parent", "in", subset).stream()

    for doc in usable_shards:
        bulkwriter.update(doc.reference, { "Usable": True})

    subset_start_index = subset_start_index + 10

bulkwriter.flush()

In [ ]:
bulkwriter.close()